# Tutorium 5

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	minimiere\ \ Z = \sum^{I}_{i=1} f_{i} \cdot y_{i} + \sum^{I}_{i=1} \sum^{J}_{j=1} c_{ij} \cdot x_{ij}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&& \sum^{I}_{i=1} x_{ij} &= d_{j} && \forall j \in J \\[5pt]
&& \sum^{J}_{j=1} x_{ij} &\leq b_{i} \cdot y_{i} && \forall i \in I \\[10pt]
&& x_{ij} &\geq 0 && \forall i \in I, \forall j \in J \\[5pt]
&& y_{i} &\in \left\{ 0, 1 \right\} &&
\end{align}


## Aufgabe 5b)
Lesen Sie aus den csv-Dateien die erforderlichen Daten ein inkl. der Namen der Absatzorte und Standorte und deren Koordinaten.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import csv

### Absatzorte.csv

| Ort                        | Bedarf | xKoordinate | yKoordinate | 
|:---------------------------|--------|-------------|-------------| 
| Flat Iron Building         | 336    | 30          | 41          | 
| Empire State Building      | 598    | 33          | 50          | 
| Chrysler Building          | 314    | 41          | 53          | 
| Central Park               | 425    | 49          | 88          | 
| One World Trade Center     | 468    | 10          | 10          | 
| Metropolitan Museum of Art | 308    | 52          | 84          | 
| Rockefeller Center         | 545    | 38          | 61          | 
| Guggenheim Museum          | 548    | 55          | 88          | 
| Times Square               | 350    | 32          | 58          | 
| Apollo Theater             | 360    | 62          | 118         | 

Hinweis: Es bietet sich an die x,y-Koordinaten jedes Ortes als Dictionary mit den Keys "X" und "Y" zu speichern. Alternativ kann auch eine Liste oder Tupel gewählt werden.

In [ ]:
d=[]
ao=[]
posAo=[]
with open("Absatzorte.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.DictReader(csv_file)
     for row in csv_reader:
          d.append(int(row["Bedarf"]))
          ao.append(row["Ort"])
          posAo.append({"X" : int(row["xKoordinate"]), "Y" : int(row["yKoordinate"])})

### Standorte.csv

| Ort | Kapazität | Fixkosten | xKoordinate | yKoordinate | 
|-----|-----------|-----------|-------------|-------------| 
| A   | 2211      | 88440     | 31          | 42          | 
| B   | 425       | 17000     | 35          | 60          | 
| C   | 850       | 34000     | 45          | 70          | 
| D   | 1276      | 51040     | 60          | 100         | 

In [ ]:
b=[]
f=[]
so=[]
posSo=[]
with open("Standorte.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.DictReader(csv_file)
     for row in csv_reader:
          b.append(int(row["Kapazität"]))
          f.append(int(row["Fixkosten"]))
          so.append(row["Ort"])
          posSo.append({"X" : int(row["xKoordinate"]), "Y" : int(row["yKoordinate"])})

## Aufgabe 5c)
Berechnen Sie die Transportkosten mit Hilfe der Distanzen nach der Manhattan-Metrik. Nutzen Sie dabei die Funktion `abs()` und gehen Sie von einem km-Kostensatz von 2.5 GE/km aus.

### Definition der Mengen

In [ ]:
I_max = len(b)
J_max = len(d)

I = range(I_max)
J = range(J_max)

### Berechnung der Kosten

In [ ]:
costPerKm = 2.5

In [ ]:
c = []
for i in I:
     newRow = []
     for j in J:
          distance = abs(posSo[i]["X"] - posAo[j]["X"]) + abs(posSo[i]["Y"] - posAo[j]["Y"])
          newRow.append(costPerKm * distance)
     c.append(newRow)

## Aufgabe 5d)
Lassen Sie sich zur Kontrolle jede Zeile der berechneten Kostenmatrix in der Konsole ausgeben und lösen Sie das Problem.

In [ ]:
for row in c:
     print(row)

Initialisierung des Modells:

In [ ]:
m = gp.Model()

Initialisierung der Variablen:

In [ ]:

x = {}
for i in I:
     for j in J:
          x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i)+str(j))
y = {}
for i in I:
     y[i] = m.addVar(vtype=GRB.BINARY, name="y_"+str(i))

Definition der Zielfunktion:

In [ ]:
m.setObjective(gp.quicksum(f[i]*y[i] for i in I) + gp.quicksum(c[i][j]*x[i,j] for j in J for i in I), GRB.MINIMIZE)

Hinzufügen der Nebenbedingungen:

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) == d[j], "meet_demand_" + str(j))

for i in I:
     m.addConstr(gp.quicksum(x[i,j] for j in J) <= b[i] * y[i], "meet_prod_" + str(i))

Optimierung:

In [ ]:
m.optimize()

## Aufgabe 5e)
Lassen Sie sich die Ergebnisse mit der Konsolenausgabe aus 3e) -- 3h) ausgeben. Welche Besonderheiten lassen sich feststellen?

In [ ]:
print("\nOptimale Standortwahl bei mehreren Betriebsstätten\n")

print("Die gesamten Kosten des Transportes betragen", m.getAttr(GRB.Attr.ObjVal), "GE.")

for i in I:
     if y[i].X > 0:
          print("In", so[i], "wird ein Standort errichtet.")
          print("\tFixkosten:", f[i])
          print("\tVariable Kosten:", sum([c[i][j] * x[i,j].X for j in J]))
     else:
          print("In", so[i], "wird KEIN Standort errichtet.")


for i in I:
     for j in J:
          if x[i,j].X > 0:
               print("Von", so[i], "werden", x[i,j].X, "ME\tnach", ao[j], "geliefert, die Transportkosten betragen dabei", c[i][j] * x[i,j].X, "GE.")

Besonderheiten:
* Sehenswürdigkeiten werden von verschiedenen Standorten beliefert

## Aufgabe 5f)


In [ ]:
for i in I:
     constr = m.getConstrByName("meet_prod_" + str(i))
     print("Nebenbedingung", constr.ConstrName, "hat einen Schlupf von", constr.Slack)

Auswertung:
* Standort 0 bzw. A könnte noch 85 weitere Einheiten ausliefern.
* Standort 1 bzw. B hat keine weitere Einheiten übrig, da dieser Standort nicht errichtet wird.
* Standort C und D transportieren alle verfügbaren Einheiten.

## Aufgabe 5g)

In [ ]:
costPerKm = 25

Auswertung:
* Alle Standorte werden errichtet.
* Standort A und D haben noch Einheiten übrig, die nicht transportiert werden.

## Aufgabe 5h)


In [ ]:
m.write("model.lp")
m.write("solution.sol")